In [1]:
import pandas as pd
import os

# Пути к файлам и соответствующие названия колонок
file_paths = {
    'bert': '/bert/sim-words5-bert.csv',
    'glove': '/glove_python/sim-words10-glove.csv',
    'pmi': '/pmi/similar_words_pmi.csv',
    'word2vec': '/word2vec/similar_words10.csv'
}

# Создаем пустой DataFrame с колонкой 'word'
data = pd.DataFrame(columns=['word'])

# Обрабатываем каждый файл
for model_name, path in file_paths.items():
    # Читаем CSV-файл
    full_path = ".." + path
    df = pd.read_csv(full_path)
    
    # Переименовываем колонку Most_Similar_Word в название модели
    df = df.rename(columns={'Most_Similar_Word': model_name})
    
    # Если это первый файл, используем его слова как основу
    if data.empty:
        data['word'] = df['Word']
    
    # Добавляем данные в основной DataFrame
    data[model_name] = df[model_name]

In [1]:
from gensim.models import KeyedVectors, Word2Vec

# Загрузка эталонной модели
thesaurus = KeyedVectors.load_word2vec_format("thesaurus.w2v", binary=False, encoding='latin1')
# thesaurus = Word2Vec.load("thesaurus.w2v", encoding='latin1')

ValueError: could not convert string to float: 'ö(\x83;¤Ð\x90;\x1f\x05{»{ÔV»¸\x1e³:qýE;×£v:\x9a\x99]9ö(l»×c\x11;\x00À\x8e»®Ç£:ÃõDº\x85ëzºÍLY»3³öº='

In [8]:
with open("thesaurus.w2v", "rb") as f:
    first_lines = [f.readline() for _ in range(2)]
    print(first_lines)

[b'1239965 100\n', b'</s> \xf6(\x83;\xa4\xd0\x90;\x1f\x05{\xbb{\xd4V\xbb\xb8\x1e\xb3:q\xfdE;\xd7\xa3v:\x9a\x99]9\xf6(l\xbb\xd7c\x11;\x00\xc0\x8e\xbb\xae\xc7\xa3:\xc3\xf5D\xba\x85\xebz\xba\xcdLY\xbb3\xb3\xf6\xba=\n']
